# Graphs

Graphs are one of the most common data structures in computer science;
graph-based modeling of problems is at the heart of many systems we use
every day, such as routing in Google Maps or recommending friends on Facebook.

[Social Network Analysis](https://en.wikipedia.org/wiki/Social_network_analysis)
(SNA) is a branch of sociology that explores social structures through the use
of analytical tools, such as graphs. In this assignment, you will implement
basic social network analysis algorithms on a graph extracted from GitHub.

You can find the graph data [at this link](github.graph). The data looks like
this (CSV format):

```
follower,followed
1570,9236
9236,1570
13256,9236
9236,13256
13256,1570
1570,13256
```

If we take the first line, it means that user `1570` follows user `9236`.

## Loading the graph

**T (10 points):** Write a function that takes as input a file name and
loads the data into an adjacency list representation.

In [5]:
def load(graph_file='github.csv'):
    """
    Loads the data from the file in the provided argument into an in-memory
    graph (as an adjacency list)
      """
    graph_data = open(graph_file)
    
    graph = list()
    
    for line in graph_data:
        pair = line.split(',')
        
        existing_node = [(node, neighbours) for (node, neighbours) in graph if node == int(pair[0].strip())]
        if len(existing_node) == 1:
            existing_node[0][1].append(int(pair[1].strip()))
        else:
            graph.append((int(pair[0].strip()), [int(pair[1].strip())]))
            
        existing_node = [(node, neighbours) for (node, neighbours) in graph if node == int(pair[1].strip())]
        if len(existing_node) == 1:
            existing_node[0][1].append(int(pair[0].strip()))
        else:
            graph.append((int(pair[1].strip()), [int(pair[0].strip())]))
    
    return sorted(graph)

Test your implementation:

In [3]:
graph = load()
for node in graph:
    print(node)

(92, [34575, 2907, 59570, 34655, 4140, 24553, 59570, 1567, 4140, 1569, 150, 17888, 10967, 465, 7580, 26457, 5810])
(93, [2907, 2812, 6145, 44451, 3029, 2618])
(150, [143808, 37207, 36671, 2907, 86717, 92, 29166, 143808, 197069, 152772, 75784, 103533, 111065, 198885, 152772, 233097, 141830, 1567, 11726, 2576, 11149, 6985, 2556, 17407, 13256, 6891, 14006, 47281, 563808, 13823, 13823, 4620, 4620, 5274, 67159, 67159, 198581, 479813, 38066, 30453, 22596, 160493])
(306, [8267, 41223, 9236, 33018, 26395, 1063, 1934, 167231, 11154, 44990])
(344, [2907, 7580, 61272, 61272, 35469, 35469, 6569, 187852])
(346, [7547, 59216, 1570, 3892, 5995, 96349, 1563, 1063, 52402, 3063, 1934, 2146, 2556, 8637, 2159, 16499, 36872, 64971, 78115])
(350, [6072, 6072, 176616, 33018, 3914, 134106, 82381, 209336, 134197, 134197, 132738, 278406, 55552, 4189, 4189, 11154, 2584, 358658, 4328])
(352, [2907, 1565, 14922, 14922, 1570, 2556, 7547, 16499, 3892, 1564, 9236, 7547, 3228736, 1567, 1570, 8077, 465, 34597, 10073726

From now on, you must use the graph returned by `load` in all the assignments
below.

## Basic graph metrics

**T (10 points):** Who are the 10 most connected users?

In [ ]:
def in_degree(graph, node_id):
    count = 0
    for (node, neighbours) in graph:
        for outgoing in neighbours:
            if outgoing == node_id:
                count = count + 1
            
    return count

def most_connected(graph, n = 10):
    """Returns the ids of the top-n most connected users"""
    most_connected = list()
    
    for (node, neighbours) in graph:
        connectedness = in_degree(graph, node)
#         connectedness += len(neighbours)
        
        if len(most_connected) < n:
            most_connected.append((connectedness, node))
            
        
        elif connectedness > most_connected[0][0]:
            del(most_connected[0])
            most_connected.append((connectedness, node))
            
        most_connected.sort()
    
    return [(node, connectedness) for (connectedness, node) in most_connected]
    


In [ ]:
graph = load()
n = len(graph)
most_connected_nodes = most_connected(graph, n)

total_sum = 0
for (node, connectedness) in most_connected_nodes:
    total_sum += connectedness

    
print('mean: ', total_sum / n)
median = (most_connected_nodes[int(n/2)][1] + most_connected_nodes[int(n/2+1)][1]) / 2
print('median: ', median) 

_Hint_: it helps if you first define a method called `in_degree` that calculates
the number of incoming edges in to a node.

**T (10 points):**  What is the mean and what is the median number of connections?

## Computing shortest paths

Shortest paths are the basis for many network measures. You will need to implement Dijkstra algorithm.

**T (20 points):** Write a function that computes the shortest paths
between all node pairs in the graph. 

_Hint_: Choose the appropriate shortest path algorithm for undirected graph
with no edge weights. A pair of nodes $(n_1, n_2)$ is, for our purposes,
equivalent to the pair $(n_2, n_1)$.

_Hint_: How to find all unique node pairs? Given that you create a non-duplicate
list of all your nodes, you can use Python's `itertools.combinations` function 
like so:
```{python}
from itertools import combinations
a = [1,2,3,4,5]
pairs = list(combinations(a, 2))
print pairs
```

In [63]:
def shortest_path(graph, source):
    graph_dict = dict(graph)
    
    distances = dict([(node, float('inf')) for (node, neighbours) in graph])
    prev = dict()
    distances[source] = 0
    
    Q = dict(distances)
    
    while len(Q) != 0:
        u = min(Q, key=Q.get)
        Q.pop(u)
        
        for v in graph_dict[u]:
            alt = distances[u] + 1
            
            if alt < distances[v]:
                distances[v] = alt
                prev[v] = u
    
    paths = dict([(node, None) for (node, neightbours) in graph])
    for (node, _) in paths.items():
        path = list()
        current = node
        if prev.get(current, None) != None:
            while current != source:
                path.insert(0, current)
                current = prev[current] 
            path.insert(0,source)
        
        paths[node] = path
    
    paths_distances = dict()
    for (x,y) in distances.items():
        paths_distances[(source, x)] = (y, paths.get(x, None))
        
    return paths_distances

def shortest_paths(graph):
    """
    Computes the shortest paths between any pair of nodes in the graph
  
    @return A dictionary whose keys are node pairs and values are sequences
    indicating the shortest path between the node pair.
    """

    shortest_paths = dict()
    count = 0
    for (node, neighbours) in graph:
        
        for key, value in shortest_path(graph, node).items():
            shortest_paths[key] = value
            
        count += 1
        if count % 100 == 0:
                print(count)
        
    return shortest_paths

In [66]:
shortest_paths = shortest_paths(load())

count = 0
for key, value in shortest_paths.items():
    count += 1
    if count > 100:
        break
    

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500


## Ranking important users

One of the primary uses of SNA is to identify important/influencial nodes.
A typical metric we use to quantify the importance of a node is centrality.
Several [centrality measures](https://en.wikipedia.org/wiki/Centrality) 
exist; for our purposes it is enough to calculate the **Betweeness Centrality**
of each node. The pseudocode to calculate it is given below.

To compute the betweenness of a node $n$

1. For each pair of nodes $(v1, v2)$, compute the shortest paths between them
2. For each pair of nodes $(v1, v2)$ determine the fraction of shortest paths
that include $n$
3. Sum this fraction over all pairs of vertices $(v1, v2)$

**T (30 points):** Write a function that computes the Betweenness centrality for
all nodes in the provided network

In [71]:
def betweenness(graph):
  
    betweenness = dict()
    paths = shortest_paths
    
    amount_of_paths = 0
    for ((_, _),(_,path)) in paths.items():
        if len(path) != 0:
            amount_of_paths += 1
    
    progress = 0
    for (node, _) in graph:
        
        if progress % 100 == 0:
            print(100*progress/len(graph),'%')
        
        progress += 1
        count = 0
        for ((source, end),(length,path)) in paths.items():
            if node in path:
                count += 1
        betweenness[node] = count/amount_of_paths
        
    return betweenness

**T (10 points):** Use the function above to rank the nodes (users) in
terms of importance and print 10 most important users with their importance.

In [76]:
importance = betweenness(load()).items()
importance2 = sorted(importance, key=lambda x: x[1], reverse=True)

print(importance2[:10])

[(7580, 0.13881338073376218), (1570, 0.11327569872098532), (1563, 0.07304915949831103), (465, 0.07186377984478277), (2907, 0.055715057309951875), (1063, 0.05350699717102671), (3892, 0.0526121517463044), (24452, 0.04830228398641739), (1564, 0.04587286443506864), (2556, 0.04354416341637263)]
